## This notebook demonstrates benchmarking the statement extraction pipeline by evaluating the LLM generated statements against the benchmark corpus containing correct INDRA statements. 

Import relevant modules

In [1]:
import sys
from pathlib import Path
# get current path
sys.path.append(str(Path.cwd().parent))

from indra_gpt.benchmarks.benchmark import Benchmark
from indra_gpt.resources.constants import INPUT_DEFAULT
import json
import pickle
from indra.statements.io import stmts_from_json, stmt_from_json


INFO: [2025-02-27 19:36:07] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's evaluate the performance of OpenAI model 'gpt-4o-mini', with structured_output mode enabled, and for a sample of 10 examples from the benchmark corpus. 

In [ ]:
model = 'gpt-4o-mini'
benchmark_file = INPUT_DEFAULT
structured_output = True
n_statements = 10
random_sample = False
benchmark = Benchmark(model, benchmark_file, structured_output, n_statements, random_sample)


In [3]:
benchmark_df = benchmark.get_comparison_df()


Extracting:   0%|          | 0/5 [00:00<?, ?statement/s]INFO: [2025-02-27 19:36:34] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-27 19:36:50] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-27 19:37:00] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  20%|██        | 1/5 [00:51<03:25, 51.48s/statement]INFO: [2025-02-27 19:37:04] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-27 19:37:08] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-27 19:37:17] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  40%|████      | 2/5 [01:08<01:33, 31.29s/statement]INFO: [2025-02-27 19:37:21] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO

In [ ]:
benchmark_df


,original_statement_json,generated_statements_json,original_statement,generated_statements,comparison_result,best_match_index
0,"{'type': 'Phosphorylation', 'enz': {'name': 'T...","[{'kind': 'RegulateActivity', 'type': 'Activat...","[Phosphorylation(THBS1(), EGFR(), Y, 1110)]","[Activation(THBS1(), phospholipase Cgamma(), c...","[{'equals': False, 'equals_type': False, 'agen...",0
1,"{'type': 'Ubiquitination', 'enz': {'name': 'RB...","[{'kind': 'Modification', 'type': 'Ubiquitinat...","[Ubiquitination(RBX1(bound: [KEAP1, True], bou...","[Complex(KEAP1-CUL3-RBX1 complex(), MCM2-7 com...","[{'equals': False, 'equals_type': False, 'agen...",1
2,"{'type': 'Complex', 'members': [{'name': 'CAV1...","[{'kind': 'RegulateActivity', 'type': 'Inhibit...","[Complex(CAV1(), MCL1())]","[Inhibition(CAV1(), MCL1(), degradation)]","[{'equals': False, 'equals_type': False, 'agen...",0
3,"{'type': 'Complex', 'members': [{'name': 'MCM1...","[{'kind': 'Complex', 'type': 'Complex', 'membe...","[Complex(MCM10(), POLA1())]","[Complex(DNA_polymerase_alpha(), MCM10())]","[{'equals': False, 'equals_type': True, 'agent...",0
4,"{'type': 'Acetylation', 'enz': {'name': 'KANSL...","[{'kind': 'RegulateActivity', 'type': 'Activat...","[Acetylation(KANSL1(), TP53(), K, 120)]","[Activation(MOF and MSL1v1 complex(), TP53(mod...","[{'equals': False, 'equals_type': False, 'agen...",0
